<a href="https://colab.research.google.com/github/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemyst_classification_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing Demystified | Classification with Naive Bayes
https://nlpdemystified.org<br>
https://github.com/nitinpunjabi/nlp-demystified

# spaCy upgrade and package installation.

At the time this notebook was created, spaCy had newer releases but Colab was still using version 2.x by default. So the first step is to upgrade spaCy and download a statistical model for English.
<br><br>
**IMPORTANT**<br>
If you're running this for free in the cloud rather than using a paid tier or using a local Jupyter server on your machine, then the notebook will *timeout* after a period of inactivity. If that happens and you don't reconnect in time, you will need to upgrade spaCy again and reinstall the requisite statistical package(s).
<br><br>
Refer to this link on how to run Colab notebooks locally on your machine to avoid this issue:<br>
https://research.google.com/colaboratory/local-runtimes.html

In [ ]:
!pip install -U spacy==3.*
!python -m spacy download en_core_web_sm

In [ ]:
!python -m spacy info

# First pass at building a Naive Bayes model.


As with our TF-IDF demo, we'll use the **20 newsgroups** dataset, a labelled dataset of 18,000 newsgroup posts across 20 topics.<br>
https://scikit-learn.org/stable/datasets/real_world.html#the-20-newsgroups-text-dataset

This time around, rather than fetching the posts from only one topic, we'll fetch the entire collection.

In [ ]:
# To build our model, we want the training subset only. The training
# subset is what gets downloaded by default but we explicitly
# pass the parameter here for clarity.
from sklearn.datasets import fetch_20newsgroups
training_corpus = fetch_20newsgroups(subset='train')

In [ ]:
print('Training data size: {}'.format(len(training_corpus.data)))

The training data we downloaded not only includes the posts but also a label for each post representing its topic. The posts are an array of strings while the labels are a corresponding array of numeric labels.

In [ ]:
# These are the possible topics a post can belong to.
training_corpus.target_names

In [ ]:
# These are the labels for each post.
print(training_corpus.target)

In [ ]:
# The first post along with its corresponding label.
print(training_corpus.data[0])

first_doc_label = training_corpus.target[0]
print('Label for this post: {}'.format(first_doc_label))
print('Corresponding topic: {}'.format(training_corpus.target_names[first_doc_label]))

When starting off with a dataset, it's a good idea to check its distribution. In this case, we can see at a glance this dataset is relatively balanced.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

bins, counts = np.unique(training_corpus.target, return_counts=True)
freq_series = pd.Series(counts/len(training_corpus.data))
plt.figure(figsize=(12, 8))
ax = freq_series.plot(kind='bar')
ax.set_xticklabels(bins, rotation=0)
plt.show()

Now that we have our training set, we can split it further into train and validation sets (remember the test set, in this case, is a separate download). Creating a validation set isn't always necessary. If you have a small training set like this one, you can use alternative techniques like cross-validation but we'll show a split here since we talked about it in the modelling review video. scikit-learn has a module to help us do this.<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Shuffle, then split the data into train and validation sets. Set the random_state 
# to 1 for reproducibility.
train_data, val_data, train_labels, val_labels = train_test_split(training_corpus.data, training_corpus.target, train_size=0.8, random_state=1) 
print('Training data size: {}'.format(len(train_data)))
print('Validation data size: {}'.format(len(val_data)))

Now that we have our train-validation split, let's create our spaCy tokenizer. Up to this point, we've been using the **en_core_web_sm** model.

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

By default, it comes up with a preprocessing pipeline with several components enabled. We can view these components through the *pipe_names* attribute.

In [ ]:
nlp.pipe_names

In the previous demos, we individually disabled any component we didn't need. For this first pass at building a Naive Bayes classifier, we don't need any of them; just the tokenization. Since that's the case, it's easier to instantiate a blank pipeline.<br>
https://spacy.io/api/top-level#spacy.blank

In [ ]:
nlp = spacy.blank('en')

In [ ]:
# There should be no pipeline components.
nlp.pipe_names

In [ ]:
# For this exercise, we'll remove punctuation and spaces (which
# includes newlines), filter for tokens consisting of alphabetic
# characters only, and return the token text.
def spacy_tokenizer(doc):
  return [t.text for t in nlp(doc) if \
          not t.is_punct and \
          not t.is_space and \
          t.is_alpha]

We'll vectorize using the **TfidfVectorizer**.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
%%time
vectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer)
train_feature_vects = vectorizer.fit_transform(train_data)

Scikit-learn includes a multinomial naive bayes classifier.<br>
https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

In [ ]:
from sklearn.naive_bayes import MultinomialNB

Calling *fit* on the classifier and passing it the feature vectors and corresponding labels kicks off the training.

In [ ]:
# Instantiate a classifier with the default settings.
nb_classifier = MultinomialNB()
nb_classifier.fit(train_feature_vects, train_labels)
nb_classifier.get_params()

Now that we know about the **F1 score** and have a multiclass problem, let's look at the F1 score on the training data. Since the dataset is balanced, accuracy could work here as well but we'll look at F1 since we introduced it. Scikit-learn has a module called **metrics** we can leverage. It contains a variety of scoring utilities we can use.<br>
https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics<br>
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score<br>
https://scikit-learn.org/stable/modules/model_evaluation.html#precision-recall-f-measure-metrics<br>

In [ ]:
from sklearn import metrics

In [ ]:
# Get predictions on training set and calculate F1 score.
# See documentation above for more details on what "macro" means.
train_preds = nb_classifier.predict(train_feature_vects)
print('F1 score on initial training set: {}'.format(metrics.f1_score(train_labels, train_preds, average='macro')))

So right off the bat, using simple preprocessing and vectorization, and the default settings on the Naive Bayes classifier, we get a model with a decent F1 score. This looks good, but there's a problem.<br><br>
When we downloaded the training data, we also included headers and footers which contain metadata like *subject*, and *email*.<br><br>
This can be a problem because these fields may be highly informative, causing the model to predict mostly based on the metadata rather than the post content. But if this metadata isn't available at prediction time in production, then our model is going to perform poorly.
<br><br>
So let's retrieve the training data again but without the headers, footers, and post quotes this time. Just raw post text. This makes the problem notably harder for reasons we'll see soon.

In [ ]:
# Remove headers, footers, and quotes from training set and resplit.
filtered_training_corpus = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
train_data, val_data, train_labels, val_labels = train_test_split(filtered_training_corpus.data, filtered_training_corpus.target, train_size=0.8, random_state=1) 

In [ ]:
# This is what a data point looks like now. Just plain post text.
train_data[0]

In [ ]:
# Revectorize our text and retrain our model.
%%time
train_feature_vects = vectorizer.fit_transform(train_data)
nb_classifier.fit(train_feature_vects, train_labels)

In [ ]:
# Recheck F1 score on training data.
train_preds = nb_classifier.predict(train_feature_vects)
print('F1 score on filtered training set: {}'.format(metrics.f1_score(train_labels, train_preds, average='macro')))

Now that we've removed metadata, our F1 score has dropped but still seems ok. The next step is to see how well the classifier performs on the validation set.

In [ ]:
# Vectorize the validation data.
%%time
val_feature_vects = vectorizer.transform(val_data)

In [ ]:
# Predict and evaluate.
val_preds = nb_classifier.predict(val_feature_vects)
print('F1 score on filtered validation set: {}'.format(metrics.f1_score(val_labels, val_preds, average='macro')))

That's quite a drop in F1 score. Because there are 20 classes involved, let's plot a confusion matrix to see what's going on:<br>
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html<br>
https://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Set the size of the plot.
fig, ax = plt.subplots(figsize=(15, 15))

# Create the confusion matrix. 
disp = ConfusionMatrixDisplay.from_estimator(nb_classifier, val_feature_vects, val_labels, normalize='true', display_labels=filtered_training_corpus.target_names, xticks_rotation='vertical', ax=ax)


In [ ]:
len(val_labels)

Similar to what we saw in the slides, the y-axis represents the true labels and the x-axis represents the predictions. Each square's brightness is correlated with the magnitude of its value. What we ideally want is brightness along the diagonal (top-left to bottom-right), and little to no brightness anywhere else.
<br><br>
Looking at the confusion matrix above, we can make a few observations:
1. The more specific a topic is, the better the prediction result. Baseball, hockey, and medicine are good examples. This intuitively makes sense.
2. Topics with a lot of word overlap tend to have higher errors. For example, the majority of atheism and religion.misc posts are classified under christianity. In general, the christianity column has a prevalence of brighter squares with misclassified posts from politics.misc, politics.mideast, etc.
3. There's a smaller, secondary cluster of errors around the computer-related topics (e.g. posts in electronics being misclassified as hardware).
<br><br>
Seeing the results of this matrix, at least there are plausible explanations for the discrepancies.

Let's take a look at **precision** and **recall** for each label:<br>
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html<br>
https://scikit-learn.org/stable/modules/model_evaluation.html#classification-report

In [ ]:
print(metrics.classification_report(val_labels, val_preds, target_names=filtered_training_corpus.target_names))

A few observations:
1. Atheism has a perfect precision score but terrible recall, signalling that the model was right when it classified something as under atheism, but missed the vast majority in the corpus. The model didn't classify anything under religion.misc.
2. The more specific the topic, the better it tends to do.

# Improving the model



Let's try to do better. One thing that's likely an issue is the sheer number of features we have relative to how little data there is.

In [ ]:
print('Training data size: {}'.format(len(train_data)))
print('Number of training features: {}'.format(len(train_feature_vects[0].toarray().flatten())))

So we can experiment with:
1. Removing stop words because topic identification likely depends more on keywords rather than sequences in this case.
3. Using the token lemma rather than the text.
<br>


We can't get away with the blank pipeline since we need a bunch of components to generate the lemma. So we'll load the **en_core_web_sm** model and disable named-entity recognition and parsing in the tokenizer callback.

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
unwanted_pipes = ['ner', 'parser']

# Further remove stop words and take the lemma instead of token text.
def spacy_tokenizer(doc):
  with nlp.disable_pipes(*unwanted_pipes):
    return [t.lemma_ for t in nlp(doc) if \
            not t.is_punct and \
            not t.is_space and \
            not t.is_stop and \
            t.is_alpha]

We need to re-vectorize the training set with the new tokenizer. Because there are certain components enabled, this is going to take longer (a few mins).

In [ ]:
%%time
vectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer)
train_feature_vects = vectorizer.fit_transform(train_data)

In [ ]:
# Check the number of features now.
print('Number of training features: {}'.format(len(train_feature_vects[0].toarray().flatten())))

A little better but still not great. Let's retrain our classifier and see what happens.

In [ ]:
nb_classifier.fit(train_feature_vects, train_labels)
train_preds = nb_classifier.predict(train_feature_vects)
print('Training F1 score with fewer features: {}'.format(metrics.f1_score(train_labels, train_preds, average='macro')))

Check classifier performance on validation set.

In [ ]:
%%time
val_feature_vects = vectorizer.transform(val_data)

In [ ]:
val_preds = nb_classifier.predict(val_feature_vects)
print('Validation F1 score with fewer features: {}'.format(metrics.f1_score(val_labels, val_preds, average='macro')))

We managed to squeeze out a few percentage points. Let's look at the confusion matrix and classification report.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
disp = ConfusionMatrixDisplay.from_estimator(nb_classifier, val_feature_vects, val_labels, normalize='true', display_labels=filtered_training_corpus.target_names, xticks_rotation='vertical', ax=ax)

In [ ]:
print(metrics.classification_report(val_labels, val_preds, target_names=filtered_training_corpus.target_names))

In the confusion matrix, the squares in the christian column have dimmed, signalling fewer classification errors. And although atheism now classifies better, that topic along with religion.misc remain big sources of overall errors.

Let's assume for now that we can't get or generate more data.

Next, we can try tuning a hyperparameter on the classifier. For Naive Bayes, we'll adjust the the *alpha* smoothing factor we discussed in the slides. But rather than trying a bunch ourselves, we can use a combination of **Grid Search** and **Cross Validation**.
- Grid search involves having the computer try a list of hyperparameter values for us, and returning the best performing value. This list is defined by us. Grid search is a basic technique and there are a number of other techniques such as **random search** and **bayesian optimization**.
- Cross validation is a way to evaluate machine learning models on limited datasets. It randomly splits the data into k-groups. One group is set aside as the holdout set while the classifier trains a model on the remaining groups. The resulting model is then used on the holdout group and the score recorded. This repeats itself until all groups have been used as a holdout set and an average score returned.

Scikit-learn has modules to handle both for us:<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html<br>
https://scikit-learn.org/stable/modules/grid_search.html#grid-search<br>
https://scikit-learn.org/stable/modules/cross_validation.html<br>


In [ ]:
from sklearn import model_selection

In [ ]:
# The alpha values to try.
params = {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0,],}

# Instantiate the search with the model we want to try and fit it on the training data.
multinomial_nb_grid = model_selection.GridSearchCV(MultinomialNB(), param_grid=params, scoring='f1_macro', n_jobs=-1, cv=5, verbose=5)
multinomial_nb_grid.fit(train_feature_vects, train_labels)

The resulting **GridSearchCV** object has a number of attributes you can explore:<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
<br><br>
We're interested in the best performing parameter value(s).

In [ ]:
print('Best parameter value(s): {}'.format(multinomial_nb_grid.best_params_))

You can directly access the best estimator found by the search. Let's try using it on the validation set.

In [ ]:
best_nb_classifier = multinomial_nb_grid.best_estimator_
val_preds = best_nb_classifier.predict(val_feature_vects)
print('Validation F1 score with fewer features: {}'.format(metrics.f1_score(val_labels, val_preds, average='macro')))

So we got another decent jump after using the the optimal *alpha* value. Let's look at the confusion matrix (using the best estimator so far) and classification report again.


In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
disp = ConfusionMatrixDisplay.from_estimator(best_nb_classifier, val_feature_vects, val_labels, normalize='true', display_labels=filtered_training_corpus.target_names, xticks_rotation='vertical', ax=ax)

In [ ]:
print(metrics.classification_report(val_labels, val_preds, target_names=filtered_training_corpus.target_names))

A few observations from this one:
1. Atheism and religion.misc are doing much better though still a source of errors.
2. The christian column has dimmed further in the other categories.

Given the small data size and the soft borders around various topics, what we have now is probably good enough. A few further ideas to explore:
1. Augment the training data with posts from similar subreddits.
2. Incorporate n-grams.
3. Remove the *misc* categories if your goal allows it.
4. Merge a few categories with large overlap together if your goal allows it.
5. Use the **CountVectorizer** instead of the **TfidfVectorizer**.
6. Play around with adding more stop words after seeing which ones are the most prevalent.
7. Play with the min_df, max_df, and max_features in the **TFidfVectorizer**.
8. Use a dimensionality reduction technique like Singular Value Decomposition (SVD) or dense word vectors which we'll cover in Part II.
9. Try other models: logistic regression, support vector machines, random forests, SGD classifier.

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
<br>
https://scikit-learn.org/stable/modules/svm.html#svm-classification
<br>
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
<br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html



For idea **(6)**, we can use the function below to view the most informative features.

In [ ]:
def show_top_words(classifier, vectorizer, categories, top_n):
  feature_names = np.asarray(vectorizer.get_feature_names())
  for i, category in enumerate(categories):
    prob_sorted = classifier.feature_log_prob_[i, :].argsort()[::-1]
    print("%s: %s" % (category, " ".join(feature_names[prob_sorted[:top_n]])))

In [ ]:
show_top_words(best_nb_classifier, vectorizer, filtered_training_corpus.target_names, 10)

In [ ]:
# scikit-learn comes with a dummy classifier you can compare against your model.
from sklearn.dummy import DummyClassifier

In [ ]:
# Train a dummy classifier which just guesses the most frequent class.
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(train_feature_vects, train_labels)
dummy_clf.score(val_feature_vects, val_labels)

In [ ]:
# Train a dummy classifier which just guesses a class randomly.
dummy_clf = DummyClassifier(strategy="uniform")
dummy_clf.fit(train_feature_vects, train_labels)
dummy_clf.score(val_feature_vects, val_labels)

# Creating the final Naive Bayes classifier.

Let's train the classifier we'll use on the test set. We'll use the entire original training set (including validation data) and the ideal *alpha* param.
<br>
We'll also use scikit-learn's **Pipeline** to specify a series of transformation and training steps so we can vectorize and fit a model with one call. Creating a few of these pipelines can help speed up your development and stay organized:<br>
https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html


In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
text_classifier = Pipeline([
  ('vectorizer', TfidfVectorizer(tokenizer=spacy_tokenizer)),
  ('classifier', MultinomialNB(alpha=0.01))
])

In [ ]:
%%time
text_classifier.fit(filtered_training_corpus.data, filtered_training_corpus.target)

Download the 20 newsgroups *test* dataset.

In [ ]:
filtered_test_corpus = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

The confusion matrix runs the classifier on the data so we can plot it immediately using the test data to see what's going on.

In [ ]:
%%time
fig, ax = plt.subplots(figsize=(15, 15))
ConfusionMatrixDisplay.from_estimator(text_classifier, filtered_test_corpus.data, filtered_test_corpus.target, normalize='true', display_labels=filtered_test_corpus.target_names, xticks_rotation='vertical', ax=ax)
plt.show()

Looking at the confusion matrix, we see there are still a few brighter clusters around the soft politics/religion area as well as the finer-grained computer-related topics, but overall the other topics look ok given the data we have.
<br><br>
We can leverage our pipeline to classify new documents on the fly now.
<br><br>
The function below takes a classifier, a document to classify, and an optional set of labels. It returns a tuple of the most probable class and its probability. With this information, you can choose a probability threshold over which to accept a classification. If it falls below the threshold, perhaps you can classify it in some default bucket or pass it to a human, or to another classifier downstream. You could also require a minimum string length for classification along with other conditions.
<br><br>


In [ ]:
def classify_text(clf, doc, labels=None):
  probas = clf.predict_proba([doc]).flatten()
  max_proba_idx = np.argmax(probas)
  
  if labels:
    most_proba_class = labels[max_proba_idx]
  else:
    most_proba_class = max_proba_idx

  return (most_proba_class, probas[max_proba_idx])

The strings below were taken at random from subreddits that have corresponding topics (e.g. r/space, r/cars, etc).

In [ ]:
# Post from r/medicine.
s = "Hello everyone so am doing my thesis on Ischemic heart disease have been using online articles and textbooks mostly Harrisons internal med. could u recommended me some source specifically books where i can get more about in depth knowledge on IHD."
classify_text(text_classifier, s, filtered_test_corpus.target_names)

In [ ]:
# Post from r/space.
s = "First evidence that water can be created on the lunar surface by Earth's magnetosphere. Particles from Earth can seed the moon with water, implying that other planets could also contribute water to their satellites."
classify_text(text_classifier, s, filtered_test_corpus.target_names)

In [ ]:
# Post from r/cars.
s = "New Toyota 86 Launch Reportedly Delayed to 2022, CEO Doesn't Want a Subaru Copy"
classify_text(text_classifier, s, filtered_test_corpus.target_names)

In [ ]:
# Post from r/electronics.
s = "My First Ever Homemade PCB. My SMD Soldering Skills Aren't Great, But I'm Quite Proud of it."
classify_text(text_classifier, s, filtered_test_corpus.target_names)

These are a few made-up statements with low probability which could belong to anything. In these situations, they can be dealt with as special cases.

In [ ]:
s = "I don't know if that's a good idea."
classify_text(text_classifier, s, filtered_test_corpus.target_names)

In [ ]:
s = "Hold on for dear life."
classify_text(text_classifier, s, filtered_test_corpus.target_names)

**Note:**<br>
Naive bayes is good at returning the most probable class but is regarded as a poor estimator (i.e. the actual probability values aren't very reliable) because of its naive assumption of independence, so keep that in mind.